In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize']=(20,10)
data = pd.read_csv('/kaggle/input/bengaluru-house-price-data/Bengaluru_House_Data.csv')

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.groupby('area_type')['area_type'].agg('count')

In [ ]:
data['area_type'].value_counts()

In [ ]:
data2 = data.drop(columns=['area_type','society','balcony','availability'])

In [ ]:
data2.head()

In [ ]:
data2.isnull().sum()

In [ ]:
data2=data2.dropna()

In [ ]:
data2.isnull().sum()

In [ ]:
data2.corr()

In [ ]:
data2['bhk']=data2['size'].apply(lambda x: int(x.split(" ")[0]))

In [ ]:
data2.head()

In [ ]:
data2[data2.bhk>20]

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
data2[~data2['total_sqft'].apply(is_float)].head()

In [ ]:
def con_sqrt_num(x):
    tokens = x.split('-')
    if len(tokens)==2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None


In [ ]:
data2['sqrt']=data2['total_sqft'].apply(con_sqrt_num)

In [ ]:
data2 = data2.drop(columns=['size','total_sqft'])

In [ ]:
data2.head()

In [ ]:
data2.isnull().sum()

In [ ]:
data2['sqrt'].dropna(inplace=True)

In [ ]:
data2.head()

In [ ]:
data2.isnull().sum()

In [ ]:
data2.info()

In [ ]:
data2.corr()

In [ ]:
data3 = data2.copy()

In [ ]:
data3['price_per_sqrt']= data2['price']*100000/data2['sqrt']
data3.head()

In [ ]:
data3.corr()

In [ ]:
data4 = data3.copy()

In [ ]:
data4.location = data4.location.apply(lambda x: x.strip())
location_stats = data4.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

In [ ]:
len(location_stats[location_stats<=10])

In [ ]:
location_stats_less_10 = location_stats[location_stats<=10]
location_stats_less_10

In [ ]:
data4.location = data4.location.apply(lambda x: 'other' if x in location_stats_less_10 else x )

In [ ]:
data4.head()

In [ ]:
data4[data4.sqrt/data4.bhk<300].head()

In [ ]:
data5 = data4[~(data4.sqrt/data4.bhk<300)]

In [ ]:
data5.corr()

In [ ]:
data5.price_per_sqrt.describe()

In [ ]:
def remove_outliers(df):
    df_out = pd.DataFrame()
    for key , subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqrt)
        st = np.std(subdf.price_per_sqrt)
        reduced_df = subdf[(subdf.price_per_sqrt>(m-st))& (subdf.price_per_sqrt<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
        
    return df_out    

In [ ]:
clean = remove_outliers(data5)

In [ ]:
clean.shape

In [ ]:
clean.corr()

In [ ]:
clean.location.unique()

In [ ]:
dummies = pd.get_dummies(clean.location)

In [ ]:
final = pd.concat([clean,dummies.drop('other',axis='columns')],axis='columns')

In [ ]:
final.corr()

In [ ]:
final = final.drop(columns='location')

In [ ]:
final.head(2)

In [ ]:
X = final.drop(columns='price')
X

In [ ]:
y = clean.price

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5 , test_size=0.2 , random_state=0)
cross_val_score(LinearRegression(),X,y,cv=cv)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model2  = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [ ]:
model2.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
model2.score(X_test,y_test)